In [2]:
import pandas as pd
df = pd.read_csv("training.csv")
df.head()

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3


In [3]:
df.shape

(16000, 2)

In [4]:
texts = df['text']

In [5]:
texts.dtype

dtype('O')

In [6]:
texts = texts.astype('string')
texts.dtype

string[python]

In [7]:
texts =texts.tolist()
#texts

In [8]:
from transformers import pipeline

emotion_model = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=None)

emotion_vectors = []
for text in texts:
    emotions = emotion_model(text)
    scores = {e["label"]: e["score"] for e in emotions[0]}
    emotion_vectors.append(scores)

2025-02-21 17:03:28.229102: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740146610.225397    6002 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740146610.412451    6002 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-21 17:03:33.967647: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
#emotion_vectors

In [10]:
emotion_df = pd.DataFrame(emotion_vectors)
emotion_df.head()

,sadness,disgust,anger,neutral,joy,surprise,fear
0,0.992294,0.002003,0.001669,0.001328,0.001299,0.000953,0.000454
1,0.992244,0.000391,0.001136,0.000851,0.003131,0.000624,0.001623
2,0.001159,0.000849,0.993571,0.000437,0.001093,0.000719,0.002173
3,0.216223,0.001108,0.003836,0.004859,0.762344,0.008936,0.002694
4,0.001480,0.000546,0.995492,0.000360,0.000607,0.000455,0.001060


In [11]:
df.label.value_counts()

label
1    5362
0    4666
3    2159
4    1937
2    1304
5     572
Name: count, dtype: int64

In [12]:

from sklearn.cluster import KMeans

kmeans_init = 'k-means++'

num_clusters = 6
kmeans = KMeans(n_clusters=num_clusters, init=kmeans_init, random_state=42, n_init=10)
df['Cluster'] = kmeans.fit_predict(emotion_df)

df.head(10)


,text,label,Cluster
0,i didnt feel humiliated,0,3
1,i can go from feeling so hopeless to so damned...,0,3
2,im grabbing a minute to post i feel greedy wrong,3,1
3,i am ever feeling nostalgic about the fireplac...,2,2
4,i am feeling grouchy,3,1
5,ive been feeling a little burdened lately wasn...,0,3
6,ive been taking or milligrams or times recomme...,5,0
7,i feel as confused about life as a teenager or...,4,4
8,i have been with petronas for years i feel tha...,1,2
9,i feel romantic too,2,2


In [13]:
df.Cluster.unique()

array([3, 1, 2, 0, 4, 5], dtype=int32)

In [14]:
from sklearn.metrics import classification_report

df['label'] = df['label'].astype('string') 
df.label.dtype

string[python]

In [15]:
mapping = {}
for cluster in df['Cluster'].unique():
    most_common_label = df[df['Cluster'] == cluster]['label'].mode()[0]
    mapping[cluster] = most_common_label
    
df['Predicted_Label'] = df['Cluster'].map(mapping)
df.head(20)   

,text,label,Cluster,Predicted_Label
0,i didnt feel humiliated,0,3,0
1,i can go from feeling so hopeless to so damned...,0,3,0
2,im grabbing a minute to post i feel greedy wrong,3,1,3
3,i am ever feeling nostalgic about the fireplac...,2,2,1
4,i am feeling grouchy,3,1,3
5,ive been feeling a little burdened lately wasn...,0,3,0
6,ive been taking or milligrams or times recomme...,5,0,5
7,i feel as confused about life as a teenager or...,4,4,4
8,i have been with petronas for years i feel tha...,1,2,1
9,i feel romantic too,2,2,1


In [16]:
df.Predicted_Label.unique()

array(['0', '3', '1', '5', '4'], dtype=object)

In [17]:
mapping

{3: '0', 1: '3', 2: '1', 0: '5', 4: '4', 5: '0'}

In [18]:
print("Classification Report:")
print(classification_report(df['label'], df['Predicted_Label']))


Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.93      0.91      4666
           1       0.83      0.93      0.87      5362
           2       0.00      0.00      0.00      1304
           3       0.87      0.92      0.90      2159
           4       0.87      0.91      0.89      1937
           5       0.71      0.93      0.80       572

    accuracy                           0.85     16000
   macro avg       0.69      0.77      0.73     16000
weighted avg       0.78      0.85      0.81     16000



/home/mntel/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mntel/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mntel/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
